In [1]:
import pymysql
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# db 연동
# member_mate
conn = pymysql.connect(
    user='my_srv',              # 유저 이름
    passwd='wkrldi@duqhdi12',   # 패스워드
    host='125.141.223.156',     # 호스트
    db='m_yeoboya_ai_dv',             # 데이터베이스
    charset='utf8',             # 인코딩
    port=33141                  # 포트 번호(''없이 사용)
)


cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_mate" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
mate_df = pd.DataFrame(result) # 데이터 프레임으로 변환환
mate_df.head(3)

,mem_no,mem_sex,mem_loc,mem_l_code,mem_birth_year,mem_birth_month,mem_birth_ddi,mate_slct,mate_slct_yn,mem_name_slct,...,family_conts_upd_cnt,secret_item_yn,secret_yn,respect_person_cont,my_point_cont,list_prt_yn,my_hd_item_yn,my_hd_yn,ins_date,upd_date
0,5,m,q,qnz,1977,1,9,w,y,y,...,13,y,y,,90,y,n,n,2015-07-22 16:31:35,2022-05-30 18:35:15
1,1804864,m,o,o10,1993,2,1,w,y,y,...,1,n,n,,80,y,n,n,2022-03-19 18:07:23,2022-06-06 19:12:05
2,2,m,e,e05,1971,1,3,r,y,y,...,2,n,n,,50,y,n,n,2015-07-22 16:37:25,2021-11-04 06:21:51


In [3]:
mate_df.columns

Index(['mem_no', 'mem_sex', 'mem_loc', 'mem_l_code', 'mem_birth_year',
       'mem_birth_month', 'mem_birth_ddi', 'mate_slct', 'mate_slct_yn',
       'mem_name_slct', 'mem_last_name_code', 'mem_last_name',
       'mem_first_name', 'rprsn_mov_yn', 'rprsn_mov', 'rprsn_mov_img',
       'rprsn_photo_no', 'rprsn_photo_yn', 'rprsn_photo_date', 'rprsn_photo',
       'rprsn_chrgr_name', 'photo_cnt', 'mate_conts', 'chrgr_check_req_yn',
       'check_reason', 'mate_job', 'mate_job_detail', 'mate_ann_salary',
       'mate_height', 'mate_weight', 'mate_blood', 'mate_religion', 'mate_car',
       'mate_career', 'mate_style', 'mate_charc', 'mate_hobby',
       'mate_offspring_m', 'mate_offspring_f', 'htown_loc', 'htown_l_code',
       'match_amt', 'wed_plan', 'fashion_style', 'divorce_rslt',
       'brothers_sisters', 'possess_property', 'language_skills',
       'study_abroad', 'favor_food', 'parents_slct', 'divorce_year',
       'child_raise', 'family_conts', 'smoke_slct', 'drink_slct',
       'he

In [4]:
mate_df = mate_df[['mem_no', 'mem_sex', 'mem_loc', 'mate_religion', 'mate_car', 'mate_job', 'mate_ann_salary',
       'mate_career', 'mate_style', 'mate_charc', 'mate_hobby', 'favor_food', 'possess_property',
        'smoke_slct', 'drink_slct','health_slct', 'mate_height', 'mate_slct']]

In [5]:
from collections import Counter

In [6]:
bins = [0, 150, 160, 170, 180, 200]
mate_df['mate_height'] = pd.cut(mate_df['mate_height'], bins=bins, labels=['under_150', '150', '160', '170', 'over_180'])
# Counter(mate_df['mate_height'])

In [7]:
favor_food_type = {'type_1': [5, 12, 13, 15, 92, 93, 96, 97, 98],
                   'type_2': [72, 73, 74, 75, 76, 77, 82, 83],
                   'type_3': [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46],
                   'type_4': [47, 49, 50, 52, 54, 48, 57, 63, 64, 70, 71, 94, 95, 99],
                   'type_5': [6, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
                   'type_6': [8, 9, 10, 14, 17],
                   'type_7': [51, 53, 55, 56, 59, 60, 61, 62, 65],
                   'type_8': [1, 2, 3, 4, 7, 18, 80, 100],
                   'type_9': [11, 31, 58, 66, 67, 68, 69, 79, 81, 84, 85, 86, 87, 88]}

hobby_type = {'type_1': [11, 12, 18, 40, 41, 42, 43, 44],
              'type_2': [8, 15, 16, 19],
              'type_3': [1, 3, 4, 6, 13, 14, 21, 24, 25, 26, 27, 30, 32, 33, 34, 35, 39],
              'type_4': [5, 7, 17, 28, 29]}

male_char_type = {'type_1': [3, 5, 9, 13],
                  'type_2': [1, 10, 11, 17, 19],
                  'type_3': [4, 6, 8],
                  'type_4': [2, 12, 14, 15, 16, 18]}

female_char_type = {'type_1': [1, 10, 11, 17, 19],
                    'type_2': [1, 2, 11, 12, 20, 22],
                    'type_3': [4, 5, 7, 14, 9],
                    'type_4': [16, 17, 18, 19, 21, 13]}

drink_key = {'a' : 0 ,'b' : 1, 'c' : 2, 'd' : 3}
health_key = {'a' : 0, 'b' : 1, 'c' : 2, 'd' : 3}
career_key = {0 : 1, 1: 1, 2: 1, 3 : 3, 4: 4, 5: 4, 6 : 2,7 : 2, 8 : 2, 9 : 4, 10 : 4, 11 : 4, 12 : 2}
salary_key = {1 : 1,2 : 1,3: 1,4 : 2,5: 2,6 : 3,7 : 3,8 : 3, 9 : 3,10 : 4, 13 : 5, 14 : 6,15 : 7,16 :7, 17:7, 12 : 1}
possess_key = {'b' : 1, 'c' : 1, 'd' : 1, 'e' : 2, 'f' : 3, 'g' : 4, 'h' : 5 ,'m' : 6, 'i' : 7,'j' : 7, 'k' : 7, 'l' : 7, 'n' : 7}
job_key = {1: 'c', 2: 'c', 3: 'l', 4: 'l', 5: 'n', 6: 'i', 7: 'i', 8: 'i', 9: 'j', 10: 'd', 11: 'k', 12: 'k', 13: 'n',
            14: 'h', 15: 'h', 16: 'a', 17: 'a', 18: 'g', 19: 'b', 20: 'm', 21: 'o', 22: 'f', 23: 'k', 24: 'n', 25: 'e',
            26: 'e', 27: 'n', 28: 'e', 29: 'e', 0: 'e', 30: 'o', 31: 'o', 32: 'o', 33: 'j', 34: 'o', 35: 'l', 36: 'l',
            37: 'j', 38: 'n', 39: 'j', 40: 'j', 41: 'j', 42: 'j', 43: 'n', 44: 'k', 45: 'k', 46: 'c', 47: 'j', 48: 'b',
            49: 'h', 50: 'c', 51: 'c', 52: 'c', 53: 'j', 54: 'j', 55: 'j', 56: 'j', 57: 'k', 58: 'j', 59: 'i', 60: 'i',
            61: 'i', 62: 'i', 63: 'i', 64: 'i', 65: 'i', 66: 'i', 67: 'i', 68: 'g', 69: 'g', 70: 'k', 71: 'k', 72: 'h',
            73: 'g', 74: 'h', 75: 'h', 76: 'h', 77: 'h', 78: 'h', 79: 'h', 80: 'g', 81: 'a', 82: 'a', 83: 'i', 84: 'i',
            85: 'i', 86: 'a', 87: 'n', 88: 'i', 89: 'i', 90: 'i', 91: 'i', 92: 'n', 93: 'b', 94: 'b', 95: 'b', 96: 'm',
            97: 'n', 98: 'f', 99: 'f', 100: 'f', 101: 'f', 102: 'a', 103: 'f'}

def detect_type(type_li, a_type):
    for i in type_li:
        if i in a_type:
            return 1
    else:
        return 0

def drink_code(drink) :
    result = 0
    if len(str(drink)) == 0 :
        return result
    
    if str(drink)[0].isalpha() and str(drink)[0] in drink_key.keys() :
        result = drink_key[str(drink)[0]]
    return result

def health_code(health) :
    result = 0
    if len(str(health)) == 0 :
        return result
    
    if str(health)[0].isalpha() and str(health)[0] in health_key.keys() :
        result = health_key[str(health)[0]]
    return result

def religion_code(x) :
    if str(x).isdecimal() :
        if int(x) == 5 :
            return str(0)
        elif int(x) != 2 and int(x) != 1 :
            return str(3)
        else :
            return str(x)
    else :
        return str(0)

def job_code(my_job):
    if str(my_job).isnumeric() and int(my_job) in job_key.keys():
        result = job_key[int(my_job)]
    else:
        result = job_key[1]
    return result

def career_code(c) :
    if str(c).isdecimal() and int(c) in career_key.keys():
        result = career_key[int(c)]
    else :
        result = 3
    return result

def salary_code(sal) :
    if str(sal).isdecimal() and int(sal) in salary_key.keys() :
        result = salary_key[int(sal)]
    else :
        result = 1
    return result

def pos_code(pos) :
    if str(pos).isalpha() and pos in possess_key.keys() :
        result = possess_key[pos]
    else :
        result = 1
    return result

In [8]:
style_f_key = {
1:'평범함',
2:'청순함',
3:'귀여움',
4:'섹시함',
5:'날씬함',
6:'통통함',
7:'뚱뚱함',
8:'깔끔함',
9:'털털함',
10:'꽃미녀',
11:'패션왕',
12:'시크함',
13:'뇌순녀',
14:'뇌섹녀', 
}

style_m_key = {
1:'평범함',
2:'귀여움',
3:'섹시함',
4:'꽃미남',
5:'날씬함',
6:'듬직함',
7:'뚱뚱함',
8:'깔끔함',
9:'털털함',
10:'',
11:'패션왕',
12:'시크함',
13:'뇌순남',
14:'뇌섹남',
15:'근육질',
16:'건장함'
   
}

def style_code(style, sex) :
    if style.isdecimal() :
        if sex == 'm' and style in style_m_key.keys() :
            return str(style)
        elif sex == 'f' and style in style_f_key.keys() :
            return str(style)
        else :
            return '1'
    else :
        return '1'

In [9]:
mate_df['drink_slct'] = mate_df['drink_slct'].apply(drink_code)
mate_df['smoke_slct'] = mate_df['smoke_slct'].apply(lambda x : 1 if len(str(x)) > 0 and str(x)[0] == 'c' else 0)
mate_df['health_slct'] = mate_df['health_slct'].apply(health_code)

In [10]:
def char_transform(df):
    df['mate_charc'] = df['mate_charc'].apply(lambda x: [int(x) for x in x.split(',') if x.isdecimal()])
    for i in range(1, 5) :
        df['char_type_'+str(i)] = df.apply(lambda x : detect_type(globals()['female_char_type']['type_'+str(i)],x['mate_charc']) if x['mem_sex'] == 'f' else detect_type(globals()['male_char_type']['type_'+str(i)], x['mate_charc']), axis=1)
    return df

mate_df = char_transform(mate_df)

In [11]:
def food_transform(df):
    df['favor_food'] = df['favor_food'].apply(lambda x: [int(i) for i in str(x).split(',') if i.isdecimal()])
    for i in range(1, 10) :
        df['food_type_'+str(i)] = df['favor_food'].apply(lambda x : detect_type(globals()['favor_food_type']['type_'+str(i)], x))
    return df

mate_df = food_transform(mate_df)

In [13]:
def hobby_transform(df) :
    df['mate_hobby'] = df['mate_hobby'].apply(lambda x: [int(i) for i in str(x).split(',') if i.isdecimal()])
    for i in range(1, 5) :
        df['hobby_type_'+str(i)] = df['hobby_type'].apply(lambda x : detect_type(globals()['hobby_type']['type_'+str(i)], x))
    return df

mate_df = hobby_transform(mate_df)

In [14]:
mate_df['mate_religion'] = mate_df['mate_religion'].apply(religion_code)
mate_df['mate_job'] = mate_df['mate_job'].apply(job_code)
mate_df['mate_career'] = mate_df['mate_career'].apply(career_code)
mate_df['mate_car'] = mate_df['mate_car'].apply(lambda x: 0 if x == 9 else 1)
mate_df['mate_ann_salary'] = mate_df['mate_ann_salary'].apply(salary_code)
mate_df['possess_property'] = mate_df['possess_property'].apply(pos_code)

In [15]:
mate_df['mate_style'] = mate_df.apply(lambda x : style_code(x['mate_style'], x['mem_sex']), axis=1)

In [16]:
del mate_df['mate_charc'], mate_df['favor_food'], mate_df['mate_hobby'], mate_df['mem_sex']

In [17]:
df = pd.get_dummies(mate_df)
df.columns

Index(['mem_no', 'mate_car', 'mate_ann_salary', 'mate_career',
       'possess_property', 'smoke_slct', 'drink_slct', 'health_slct',
       'char_type_1', 'char_type_2', 'char_type_3', 'char_type_4',
       'food_type_1', 'food_type_2', 'food_type_3', 'food_type_4',
       'food_type_5', 'food_type_6', 'food_type_7', 'food_type_8',
       'food_type_9', 'hobby_type_1', 'hobby_type_2', 'hobby_type_3',
       'hobby_type_4', 'mem_loc_a', 'mem_loc_b', 'mem_loc_c', 'mem_loc_d',
       'mem_loc_e', 'mem_loc_f', 'mem_loc_g', 'mem_loc_h', 'mem_loc_i',
       'mem_loc_j', 'mem_loc_k', 'mem_loc_l', 'mem_loc_m', 'mem_loc_n',
       'mem_loc_o', 'mem_loc_p', 'mem_loc_q', 'mem_loc_y', 'mem_loc_z',
       'mate_religion_0', 'mate_religion_1', 'mate_religion_2',
       'mate_religion_3', 'mate_job_a', 'mate_job_b', 'mate_job_c',
       'mate_job_d', 'mate_job_e', 'mate_job_f', 'mate_job_g', 'mate_job_h',
       'mate_job_i', 'mate_job_j', 'mate_job_k', 'mate_job_l', 'mate_job_m',
       'mate_job_n'

In [18]:
# member_pf_view_all_log
cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_pf_view_all_log where ins_date between '2022-05-22' and '2022-05-28'" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
view_df = pd.DataFrame(result) # 데이터 프레임으로 변환환
view_df.head(3)

,auto_no,mem_no,mem_sex,ptr_mem_no,ptr_mem_sex,ptr_rprsn_photo_yn,open_slct,open_date,open_type,item_cnt,ins_date
0,329457069,1852521,m,1600821,f,n,v,2022-05-22 00:00:00,0,0,2022-05-22 00:00:00
1,329457070,1802892,m,1852768,f,n,v,2022-05-22 00:00:04,0,0,2022-05-22 00:00:04
2,329457071,1844756,m,1851181,f,n,v,2022-05-22 00:00:04,0,0,2022-05-22 00:00:04


In [19]:
# 1개만 카운트하기 위해 중복 회원 쌍 제거, member_mate에 있는 회원만 사용
view_df.drop_duplicates(inplace=True, subset=['mem_no','ptr_mem_no'])
view_df.reset_index(drop=True,inplace=True)
view_df = view_df[view_df['mem_no'].isin(mate_df['mem_no'].values)]
view_df = view_df[view_df['ptr_mem_no'].isin(mate_df['mem_no'].values)]
print(len(view_df))

953029


In [20]:
# member_msg_log_back
cursor = conn.cursor(pymysql.cursors.DictCursor)
sql = "select * from m_yeoboya.member_msg_log_back where ins_date between '2022-05-22' and '2022-05-28'" 
cursor.execute(sql)
result = cursor.fetchall()
cursor.close()
msg_df = pd.DataFrame(result) # 데이터 프레임으로 변환환
msg_df.head(3)

,auto_no,mem_no,mem_sex,mem_birth_year,mate_slct,login_media,mem_slct,ptr_mem_no,ptr_mem_sex,ptr_mem_birth_year,ptr_mate_slct,ptr_login_media,read_yn,msg_type,msg_slct,msg_cont,msg_etc,file_name,file_size,ins_date
0,202459354,1766800,m,1981,w,s,o,1852683,f,1979,w,s,n,1,0,서로 적극적으로 해야,"{""mediaFileName"":"""",""mediaPlayTime"":"""",""mediaP...",,,2022-05-22 00:00:00
1,202459356,1852617,m,1978,w,s,o,1852148,f,1974,r,,n,1,0,반갑습니다^^,"{""mediaFileName"":"""",""mediaPlayTime"":"""",""mediaP...",,,2022-05-22 00:00:01
2,202459358,509026,m,1987,w,s,n,1832078,f,1993,w,s,n,1,0,재밌겠어요 그죠,"{""mediaFileName"":"""",""mediaPlayTime"":"""",""mediaP...",,,2022-05-22 00:00:01


In [21]:
# 1개만 카운트하기 위해 중복 회원 쌍 제거, member_mate에 있는 회원만 사용
msg_df.drop_duplicates(inplace=True, subset=['mem_no','ptr_mem_no'])
msg_df = msg_df[msg_df['mem_no'].isin(mate_df['mem_no'].values)]
msg_df = msg_df[msg_df['ptr_mem_no'].isin(mate_df['mem_no'].values)]
msg_df.reset_index(drop=True,inplace=True)
len(msg_df)

146138

In [22]:
# x가 열람한 회원 리스트
df['view_list'] = None
for i in tqdm(range(len(df))) :
    view_list = view_df[view_df['mem_no']==df.loc[i, 'mem_no']]['ptr_mem_no'].values.tolist()
    df.loc[i, 'view_list'] = [[view_list]]
# df.head(3)

100%|█████████████████████████████████████████████████████████████████████████| 124326/124326 [08:18<00:00, 249.42it/s]


In [23]:
df['view_list'] = df['view_list'].apply(lambda x : x[0])
df['view_count'] = df['view_list'].apply(lambda x : len(x)) # 총 열람 횟수

In [24]:
# x를 열람한 회원 리스트
df['viewed_list'] = None
for i in tqdm(range(len(df))) :
    viewed_list = view_df[view_df['ptr_mem_no']==df.loc[i, 'mem_no']]['mem_no'].values.tolist()
    df.loc[i, 'viewed_list'] = [[viewed_list]]

100%|█████████████████████████████████████████████████████████████████████████| 124326/124326 [08:35<00:00, 241.05it/s]


In [25]:
df['viewed_list'] = df['viewed_list'].apply(lambda x : x[0])
df['viewed_count'] = df['viewed_list'].apply(lambda x : len(x)) # 총 열람된 횟수

In [26]:
# 메시지 발신 회원 리스트
df['send_list'] = None
for i in tqdm(range(len(df))) :
    send_list = msg_df[msg_df['mem_no']==df.loc[i, 'mem_no']]['ptr_mem_no'].values.tolist()
    df.loc[i, 'send_list'] = [[send_list]]

100%|█████████████████████████████████████████████████████████████████████████| 124326/124326 [07:25<00:00, 279.23it/s]


In [27]:
# 메시지 수신 회원 리스트
df['sent_list'] = None
for i in tqdm(range(len(df))) :
    sent_list = msg_df[msg_df['ptr_mem_no']==df.loc[i, 'mem_no']]['mem_no'].values.tolist()
    df.loc[i, 'sent_list'] = [[sent_list]]

100%|█████████████████████████████████████████████████████████████████████████| 124326/124326 [06:35<00:00, 314.07it/s]


In [28]:
df['send_list'] = df['send_list'].apply(lambda x : x[0])
df['sent_list'] = df['sent_list'].apply(lambda x : x[0])

In [29]:
df['send_count'] = df['send_list'].apply(lambda x : len(x)) # 총 발신 수 
df['sent_count'] = df['sent_list'].apply(lambda x : len(x)) # 총 수신 수

In [30]:
mate_df.columns[1:]

Index(['mem_loc', 'mate_religion', 'mate_car', 'mate_job', 'mate_ann_salary',
       'mate_career', 'mate_style', 'possess_property', 'smoke_slct',
       'drink_slct', 'health_slct', 'mate_height', 'mate_slct', 'char_type_1',
       'char_type_2', 'char_type_3', 'char_type_4', 'food_type_1',
       'food_type_2', 'food_type_3', 'food_type_4', 'food_type_5',
       'food_type_6', 'food_type_7', 'food_type_8', 'food_type_9',
       'hobby_type_1', 'hobby_type_2', 'hobby_type_3', 'hobby_type_4'],
      dtype='object')

In [35]:
def get_explanation(x, y, k=3) :
    view_list = df[df['mem_no']==x].view_list.values[0]
    send_list = df[df['mem_no']==x].send_list.values[0]
    
    # m_x
    m_x = [1 if i in send_list else 0 for i in view_list]
    a_v_df = pd.DataFrame([m_x], columns=view_list, index=[x]).T

    # S_x_a_v
    for col in mate_df.columns[1:] :
        s_x_a_v = [mate_df[mate_df['mem_no']==y][col].values[0] for y in a_v_df.index]
        a_v_df[col] = s_x_a_v
        
    a_v_df = pd.get_dummies(a_v_df)

    y_a_v = pd.get_dummies(mate_df[mate_df['mem_no']==y])
    
    for y_col in y_a_v.columns :
#         print(y_col)
        if y_col in a_v_df.columns and y_a_v[y_col].values[0] != 1 :
            a_v_df.drop(y_col, inplace=True, axis=1)
#             print(f'{y_col} removed')
    a_v_df.drop([c for c in a_v_df.columns if c not in y_a_v.columns and c != x], axis=1, inplace=True)

    
    result = a_v_df.corr()[x].sort_values(ascending=False)
    print(result)
    print(f'Highest correlation features for {x} : {[result.index[1]]}, correlation value : {result.values[1]}')
    return result.index[1:k+1].tolist()

In [36]:
get_explanation(1855569, 1737706)

         1855569
1245721        0
1725059        0
1808458        0
1839393        0
1517227        0
...          ...
887277         0
1794704        0
1187272        0
1312175        0
1292879        0

[182 rows x 1 columns]
1855569            1.000000
mate_height_170    0.174061
drink_slct         0.132921
mate_job_c         0.088240
food_type_4        0.066949
char_type_1        0.025921
char_type_3       -0.027176
mate_religion_0   -0.038211
mate_slct_w       -0.044057
mem_loc_e         -0.055945
smoke_slct        -0.140573
mate_car                NaN
mate_style_1            NaN
Name: 1855569, dtype: float64
Highest correlation features for 1855569 : ['mate_height_170'], correlation value : 0.17406130986843177


['mate_height_170', 'drink_slct', 'mate_job_c']

In [34]:
get_explanation(1737706, 1855569)

1737706            1.000000
mate_religion_3    0.083355
mate_height_160    0.006136
mate_job_j         0.005096
food_type_3       -0.029494
hobby_type_2      -0.065713
drink_slct        -0.065770
food_type_2       -0.089455
food_type_1       -0.092933
food_type_5       -0.114587
hobby_type_3      -0.122824
food_type_4       -0.141781
hobby_type_4      -0.171808
mate_slct_r       -0.205411
mem_loc_i         -0.213970
mate_car                NaN
mate_style_1            NaN
Name: 1737706, dtype: float64
Highest correlation features for 1737706 : ['mate_religion_3'], correlation value : 0.08335544402616608


['mate_religion_3', 'mate_height_160', 'mate_job_j']